#### B站临床预测模型参考：
https://www.bilibili.com/video/BV1TDRGYfEVm/?spm_id_from=333.1387.collection.video_card.click&vd_source=fe18fc7331f8146ed53da453b38c3226

#### 其他官方实现代码：
- scikit-learn中文社区：https://scikit-learn.org.cn/
- 【布客】sklearn 中文翻译：https://sklearn.apachecn.org/


--------------------
代码目标是实现机器学习模型的训练、调优、评估以及结果的可视化分析。
项目主要包括以下几个步骤：
- 数据加载与预处理：
首先从Excel文件中加载了包含多特征的医学数据，并将其存储在df中，使用LightGBM模型进行特征的重要性排序。通过训练LGBM模型并获取特征重要性（feature_importances_），对特征进行排序，并选取前30个最重要的特征，最后根据LGBM模型的特征重要性，选择了前8个特征进行模型训练。这些特征是：X30.X 39.X 46.X 32, X 34，X 33.X 9. X 28.目标变量是y（即分类标签），并将特征和目标变量分开存储，进行后续的模型训练
- 模型训练与调优：
使用多种机器学习模型进行训练，包括人工神经网络（ANN）、决策树（DT）、极限随机树（ET）、梯度提升机（GBM）、K近邻（KNN）、LightGBM（LGBM）、随机森林（RF）、支持向量机（SVM）和XGBoost（XGB）。为每个模型定义了不同的超参数，并通过GridSearchCV进行网格搜索，找到最优的参数配已置，每个模型使用5折交叉验证来评估其性能，确保模型在不同数据子集上的表现稳定
- 模型评估与性能对比：
对每个模型进行评估，计算常见的分类指标，包括准确率、敏感性（召回率）、特异性、精确度（阳性预测值）、负性预测值、F1分数以及Kappa系数等，绘制多个图表进行模型性能对比，包括ROC曲线、Precision-Recall曲线等，以展示不同模型在训练集和测试集上的表现
- 模型结果的可视化：
对最优模型（在这个例子中是ExtraTreesClassifier）进行了深入的可视化分析。通过SHAP值解释模型的决策过程，展示特征的重要性和对每个样本预测的影响，使用PCA进行降维，将数据投影到一维空间，并绘制散点图展示每个数据点在降维空间中的分布，同时根据预测的概率值对数据点进行颜色编码，区分不同类别，绘制了多个SHAP图（如条形图、散点图、瀑布图等），这些图表帮助解释模型的决策依据和各特征对模型预测的影响

In [ ]:
# 方法一：独热编码（One-Hot Encoding）
# 独热编码是一种将类别数据转换为数值型数据的方法。对于每个类别数据，我们为其创建一个新的二进制列，并标记该列的值为1。其他列的值均为0。这样，每个类别的唯一值都会被表示为一个独热向量。
# 例如，假设有一个名为“月份”的列，包含以下类别数据：’1月’、’2月’、’3月’、’4月’、’5月’、’6月’、’7月’、’8月’、’9月’、’10月’、’11月’和’12月’。
# 我们可以为每个月份创建一个二进制列，例如：
import pandas as pd
from sklearn.preprocessing import LabelEncoder
# 示例数据
data = {'month': ['1月', '2月', '3月', '4月', '5月', '6月', '7月', '8月', '9月', '10月', '11月', '12月'], 'value': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]}
df = pd.DataFrame(data)
# 使用LabelEncoder进行标签编码
le = LabelEncoder()
le_fit = le.fit(df['month']) # Fit the LabelEncoder to the month column data.
label_encoded = le_fit.transform(df['month']) # Transform the month column data.
print(label_encoded)

In [ ]:
# 方法二：标签编码（Label Encoding）
# 标签编码是一种简单地将类别数据转换为数值型数据的方法。它为每个唯一类别分配一个整数。通常用于类别数量不多的情况。
# Python代码实现如下：
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
# 示例数据
data = {'month': ['1月', '2月', '3月', '4月', '5月', '6月', '7月', '8月', '9月', '10月', '11月', '12月'], 'value': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]}
df = pd.DataFrame(data)
# 使用OneHotEncoder进行独热编码
ohe = OneHotEncoder(sparse_output=False)
ohe_fit = ohe.fit(df[['month']])
one_hot_encoded = ohe_fit.transform(df[['month']])
print(one_hot_encoded)

In [ ]:
#### 0.设置工作路径
import os

# 设置工作路径
os.chdir("/home/phl/PHL/Car-T/model-1")

# 获取并打印当前工作路径
current_path = os.getcwd()
print("当前工作路径：", current_path)

In [ ]:
### 针对动态数据特定时间窗口（Day 0-14）的缺失值分析代码（gemini生成）

# 锁定时间窗口：仅筛选 0 <= Day <= 14 的数据。
# 计算覆盖率：统计每个指标在该窗口内有多少患者拥有至少一次有效观测。
# 评估缺失率：基于总患者数计算缺失比例，并根据 20% 的阈值给出保留/剔除建议。

# 动态数据缺失值分析
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

# =============================================================================
# 1. 设置路径与加载基础数据
# =============================================================================
# 引用 split.ipynb 中的路径配置
static_path = "/home/phl/PHL/Car-T/model-1/B-NHL_reindexed_example/csv/B-NHL_static_data.csv"
dynamic_dir = "/home/phl/PHL/Car-T/model-1/B-NHL_reindexed_example/processed"

# 读取静态数据以获取合法的患者 ID 列表
df_static = pd.read_csv(static_path)

# 复用 split.ipynb 的清洗逻辑，确保分析的患者群体与后续建模一致
# 剔除 AAS (分期) 为空的患者
if 'AAS' in df_static.columns:
    df_static = df_static.dropna(subset=['AAS'])
# 剔除 Infection (目标变量) 为空的患者
if 'Infection' in df_static.columns:
    df_static = df_static.dropna(subset=['Infection'])

valid_patient_ids = df_static['ID'].unique()
total_patients = len(valid_patient_ids)
print(f"分析纳入患者总数: {total_patients}")

# =============================================================================
# 2. 动态数据缺失值分析逻辑
# =============================================================================
# 分析参数设置
START_DAY = -15
END_DAY = 14
MISSING_THRESHOLD = 0.20  # 20% 缺失率阈值

print(f"正在分析动态数据窗口: Day {START_DAY} 至 Day {END_DAY} ...")

# 用于存储每个变量在多少个病人中出现过（至少有一次非空观测）
variable_presence_count = {} 

for pid in tqdm(valid_patient_ids):
    file_path = os.path.join(dynamic_dir, f"{pid}.csv")
    
    if os.path.exists(file_path):
        try:
            df_dyn = pd.read_csv(file_path)
            
            # 标准化时间列名处理
            time_col = None
            for col in ['Day', 'day', 'Time', 'time']:
                if col in df_dyn.columns:
                    time_col = col
                    break
            
            if time_col:
                # 1. 时间窗口聚焦：仅保留 Day 0 到 Day 14 的数据
                # 依据：排除个体差异较大的远期数据，聚焦治疗后前两周的关键期
                mask = (df_dyn[time_col] >= START_DAY) & (df_dyn[time_col] <= END_DAY)
                df_window = df_dyn[mask]
                
                if not df_window.empty:
                    # 2. 遍历该患者的所有变量
                    for col in df_window.columns:
                        # 排除 ID, 时间列 和 脏数据列(Unnamed)
                        if col in [time_col, 'ID'] or 'Unnamed' in col:
                            continue
                        
                        # 3. 检查有效性：如果该变量在这个窗口内至少有一个非空值，则视为"存在"
                        # (注：这里暂不强制要求每3-4天一次的频率，只要窗口内有数据即视为可用)
                        if df_window[col].notna().any():
                            variable_presence_count[col] = variable_presence_count.get(col, 0) + 1
                            
        except Exception as e:
            print(f"读取患者 {pid} 数据失败: {e}")

# =============================================================================
# 3. 生成分析报告
# =============================================================================
analysis_results = []

for var, count in variable_presence_count.items():
    missing_count = total_patients - count
    missing_rate = missing_count / total_patients
    
    analysis_results.append({
        'Variable': var,
        'Present_Patients': count,
        'Missing_Patients': missing_count,
        'Missing_Rate': missing_rate,
        'Status': 'Keep' if missing_rate <= MISSING_THRESHOLD else 'Drop'
    })

# 转换为 DataFrame 并排序
df_analysis = pd.DataFrame(analysis_results)
if not df_analysis.empty:
    df_analysis = df_analysis.sort_values('Missing_Rate', ascending=True)

    # 划分保留和剔除的变量
    vars_to_keep = df_analysis[df_analysis['Status'] == 'Keep']
    vars_to_drop = df_analysis[df_analysis['Status'] == 'Drop']

    print("\n" + "="*60)
    print(f"【动态变量缺失值分析报告】")
    print(f"时间窗口: Day {START_DAY} - {END_DAY}")
    print(f"缺失率阈值: {MISSING_THRESHOLD:.0%}")
    print("="*60)
    print(f"检测到的动态变量总数: {len(df_analysis)}")
    print(f"建议保留变量数 (Missing <= 20%): {len(vars_to_keep)}")
    print(f"建议剔除变量数 (Missing > 20%): {len(vars_to_drop)}")

    print("\n>>> 建议保留的高质量变量 (Top 10):")
    print(vars_to_keep[['Variable', 'Missing_Rate', 'Present_Patients']].head(10).to_string(index=False))

    print("\n>>> 建议剔除的高缺失变量 (Top 10):")
    print(vars_to_drop[['Variable', 'Missing_Rate', 'Present_Patients']].tail(10).to_string(index=False))
    
    # 提取建议保留的变量名列表，供后续使用
    suggested_features = vars_to_keep['Variable'].tolist()
    print(f"\n[代码提示] 后续特征提取时，建议仅提取以下 {len(suggested_features)} 个变量:")
    print(suggested_features)
else:
    print("未检测到任何有效的动态变量，请检查数据路径或时间窗口设置。")

In [ ]:
# 随机森林特征重要性官方实现参考：https://scikit-learn.org.cn/view/255.html（置换重要性与随机森林特征重要性(MDI)）
import matplotlib.pyplot as plt
import numpy as np

from sklearn.datasets import fetch_openml
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.inspection import permutation_importance
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

X, y = fetch_openml("titanic", version=1, as_frame=True, return_X_y=True)
rng = np.random.RandomState(seed=42)
X['random_cat'] = rng.randint(3, size=X.shape[0])
X['random_num'] = rng.randn(X.shape[0])

categorical_columns = ['pclass', 'sex', 'embarked', 'random_cat']
numerical_columns = ['age', 'sibsp', 'parch', 'fare', 'random_num']

X = X[categorical_columns + numerical_columns]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, random_state=42)

categorical_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])
numerical_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='mean'))
])

preprocessing = ColumnTransformer(
    [('cat', categorical_pipe, categorical_columns),
     ('num', numerical_pipe, numerical_columns)])

rf = Pipeline([
    ('preprocess', preprocessing),
    ('classifier', RandomForestClassifier(random_state=42))
])
rf.fit(X_train, y_train)


print("RF train accuracy: %0.3f" % rf.score(X_train, y_train))
print("RF test accuracy: %0.3f" % rf.score(X_test, y_test))


ohe = (rf.named_steps['preprocess']
         .named_transformers_['cat']
         .named_steps['onehot'])
feature_names = ohe.get_feature_names_out(input_features=categorical_columns)
feature_names = np.r_[feature_names, numerical_columns]

tree_feature_importances = (
    rf.named_steps['classifier'].feature_importances_)
sorted_idx = tree_feature_importances.argsort()

y_ticks = np.arange(0, len(feature_names))
fig, ax = plt.subplots()
ax.barh(y_ticks, tree_feature_importances[sorted_idx])
ax.set_yticklabels(feature_names[sorted_idx])
ax.set_yticks(y_ticks)
ax.set_title("Random Forest Feature Importances (MDI)")
fig.tight_layout()
plt.show()

In [ ]:
# 官方实现Spearman秩序相关性与置换重要性：https://scikit-learn.org.cn/view/254.html（具有多重共线性或相关特征的置换重要性）
from collections import defaultdict

import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import spearmanr
from scipy.cluster import hierarchy

from sklearn.datasets import load_breast_cancer
from sklearn.ensemble import RandomForestClassifier
from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split

# 首先，在乳腺癌数据集上训练随机森林，并在测试集上评估其准确性：
data = load_breast_cancer()
X, y = data.data, data.target
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)
print("Accuracy on test data: {:.2f}".format(clf.score(X_test, y_test)))

# 其次，绘制基于树的特征重要性和置换重要性。
# 置换重要性图显示，变换一个特征最多降低了0.012的准确性，这意味着没有任何一个特征是重要的。
# 这与上面计算的高精度测试是矛盾的：某些特性必须是重要的。在训练集上计算置换的重要性，以显示模型在训练过程中对每个特征的依赖程度。
result = permutation_importance(clf, X_train, y_train, n_repeats=10,
                                random_state=42)
perm_sorted_idx = result.importances_mean.argsort()

tree_importance_sorted_idx = np.argsort(clf.feature_importances_)
tree_indices = np.arange(0, len(clf.feature_importances_)) + 0.5

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 8))
ax1.barh(tree_indices,
         clf.feature_importances_[tree_importance_sorted_idx], height=0.7)
ax1.set_yticklabels(data.feature_names[tree_importance_sorted_idx])
ax1.set_yticks(tree_indices)
ax1.set_ylim((0, len(clf.feature_importances_)))
ax2.boxplot(result.importances[perm_sorted_idx].T, vert=False,
            labels=data.feature_names[perm_sorted_idx])
fig.tight_layout()
plt.show()

# 处理多线性特征
# 通过对Spearman秩序相关性执行分层聚类，选择一个阈值，并从每个聚类中保留一个特征
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 8))
corr = spearmanr(X).correlation
corr_linkage = hierarchy.ward(corr)
dendro = hierarchy.dendrogram(corr_linkage, labels=data.feature_names, ax=ax1,
                              leaf_rotation=90)
dendro_idx = np.arange(0, len(dendro['ivl']))

ax2.imshow(corr[dendro['leaves'], :][:, dendro['leaves']])
ax2.set_xticks(dendro_idx)
ax2.set_yticks(dendro_idx)
ax2.set_xticklabels(dendro['ivl'], rotation='vertical')
ax2.set_yticklabels(dendro['ivl'])
fig.tight_layout()
plt.show()

In [ ]:
# 显示对象的可视化——构造混淆矩阵和ROC曲线https://scikit-learn.org.cn/view/273.html
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

X, y = fetch_openml(data_id=1464, return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

clf = make_pipeline(StandardScaler(), LogisticRegression(random_state=0))
clf.fit(X_train, y_train)

# 使用拟合的模型，我们可以在测试数据集上计算模型的预测。 这些预测用于计算使用ConfusionMatrixDisplay绘制的混淆矩阵。
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

y_pred = clf.predict(X_test)
cm = confusion_matrix(y_test, y_pred)

cm_display = ConfusionMatrixDisplay(cm).plot()

# roc曲线需要估算器的概率或非阈值决策值。 由于逻辑回归提供了决策函数，因此我们将使用它来绘制roc曲线
from sklearn.metrics import roc_curve
from sklearn.metrics import RocCurveDisplay
y_score = clf.decision_function(X_test)

fpr, tpr, _ = roc_curve(y_test, y_score, pos_label=clf.classes_[1])
roc_display = RocCurveDisplay(fpr=fpr, tpr=tpr).plot()

# 使用预置部分中的y_score绘制精度召回曲线
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import PrecisionRecallDisplay

prec, recall, _ = precision_recall_curve(y_test, y_score,
                                         pos_label=clf.classes_[1])
pr_display = PrecisionRecallDisplay(precision=prec, recall=recall).plot()


# 将显示对象合并为一个图
import matplotlib.pyplot as plt
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 8))

roc_display.plot(ax=ax1)
pr_display.plot(ax=ax2)
plt.show()

# # 示例：绘制 SVM 的混淆矩阵官方实现：https://scikit-learn.org.cn/view/273.html
# from sklearn.metrics import confusion_matrix
# from sklearn.metrics import ConfusionMatrixDisplay

# metrics_train = calculate_metrics(svm, X_train, y_train, label='SVM')
# cm_display = ConfusionMatrixDisplay(metrics_train['Confusion_Matrix']).plot()

In [ ]:
# 带交叉验证的ROC曲线https://scikit-learn.org.cn/view/289.html
import numpy as np
import matplotlib.pyplot as plt

from sklearn import svm, datasets
from sklearn.metrics import auc
# from sklearn.metrics import plot_roc_curve  # 旧版本代码，已移除
from sklearn.metrics import RocCurveDisplay   # 新版本导入方式
from sklearn.model_selection import StratifiedKFold

# #############################################################################
# 获取数据

# 导入待处理的数据
iris = datasets.load_iris()
X = iris.data
y = iris.target
X, y = X[y != 2], y[y != 2]
n_samples, n_features = X.shape

# 增加一些噪音特征
random_state = np.random.RandomState(0)
X = np.c_[X, random_state.randn(n_samples, 200 * n_features)]

# #############################################################################
# 分类，并使用ROC曲线分析结果

# 使用交叉验证运行分类器并绘制ROC曲线
cv = StratifiedKFold(n_splits=6)
classifier = svm.SVC(kernel='linear', probability=True,
                     random_state=random_state)

tprs = []
aucs = []
mean_fpr = np.linspace(0, 1, 100)

fig, ax = plt.subplots()
for i, (train, test) in enumerate(cv.split(X, y)):
    classifier.fit(X[train], y[train])
    # === 修改开始 ===
    # 旧代码: 
#     viz = plot_roc_curve(classifier, X[test], y[test],
#                          name='ROC fold {}'.format(i),
#                          alpha=0.3, lw=1, ax=ax)

    # 新代码: 使用 RocCurveDisplay.from_estimator
    viz = RocCurveDisplay.from_estimator(
        classifier, 
        X[test], 
        y[test],
        name='ROC fold {}'.format(i),
        alpha=0.3, 
        lw=1, 
        ax=ax
    )
    # === 修改结束 ===
    interp_tpr = np.interp(mean_fpr, viz.fpr, viz.tpr)
    interp_tpr[0] = 0.0
    tprs.append(interp_tpr)
    aucs.append(viz.roc_auc)

ax.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',
        label='Chance', alpha=.8)

mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
ax.plot(mean_fpr, mean_tpr, color='b',
        label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
        lw=2, alpha=.8)

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
ax.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2,
                label=r'$\pm$ 1 std. dev.')

ax.set(xlim=[-0.05, 1.05], ylim=[-0.05, 1.05],
       title="Receiver operating characteristic example")
ax.legend(loc="lower right")
plt.show()


In [ ]:
# 验证曲线：绘制分数以评估模型https://scikit-learn.org.cn/view/116.html#
import matplotlib.pyplot as plt
import numpy as np

from sklearn.datasets import load_digits
from sklearn.svm import SVC
from sklearn.model_selection import validation_curve

X, y = load_digits(return_X_y=True)

param_range = np.logspace(-6, -1, 5)
train_scores, test_scores = validation_curve(
    SVC(), X, y, param_name="gamma", param_range=param_range,
    scoring="accuracy", n_jobs=1)
train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)
test_scores_std = np.std(test_scores, axis=1)

plt.title("Validation Curve with SVM")
plt.xlabel(r"$\gamma$")
plt.ylabel("Score")
plt.ylim(0.0, 1.1)
lw = 2
plt.semilogx(param_range, train_scores_mean, label="Training score",
             color="darkorange", lw=lw)
plt.fill_between(param_range, train_scores_mean - train_scores_std,
                 train_scores_mean + train_scores_std, alpha=0.2,
                 color="darkorange", lw=lw)
plt.semilogx(param_range, test_scores_mean, label="Cross-validation score",
             color="navy", lw=lw)
plt.fill_between(param_range, test_scores_mean - test_scores_std,
                 test_scores_mean + test_scores_std, alpha=0.2,
                 color="navy", lw=lw)
plt.legend(loc="best")
plt.show()

In [ ]:
# Lasso回归筛选变量参考：https://www.bilibili.com/video/BV1mQ4y1A7os
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV

xlsx1_filePath ="/home/phl/PHL/Car-T/model_v1/B-NHL_reindexed_example/csv/B-NHL_static_data.csv"
xlsx2_filePath ='/home/phl/PHL/Car-T/model_v1/B-NHL_reindexed_example/processed/1.csv'
data_1 = pd.read_csv(xlsx1_filePath)
data_2 = pd.read_csv(xlsx2_filePath)
rows_1, _ = data_1.shape
rows_2, _ = data_2.shape
data_1.insert(0, 'label', [0] * rows_1)
data_2.insert(0, 'label', [1] * rows_2)
data = pd.concat([data_1, data_2])
data = shuffle(data)
data = data.fillna(0)
X = data.iloc[:, 1:]
y = data['label']
ColNames = X.columns
# X = X.astype(np.float64)
X = StandardScaler().fit_transform(X)  # 新知识点
X = pd.DataFrame(X)
X.columns = ColNames


alphas = np.logspace(-3,1,50)
model_lassoCv = LassoCV(alphas = alphas,cv = 10,max_iter = 100000).fit(X,y)

print(model_lassoCv.alpha_)
coef = pd.Series(model_lassoCv.coef_, index = X.columns) # 新知识点
print("Lasso picked "+ str(sum(coef != 0))+" variables and eliminated the other "+ str(sum(coef == 0)))


index = coef[coef != 0].index
X = X[index]
# X.head()
print(coef[coef != 0])

### 前言
Boruta算法是一种用于特征选择的包裹式算法，专门设计用于确定数据集中哪些特征对预测模型是重要的
### Boruta算法原理
```
1. 构建随机森林模型：首先，Boruta算法使用随机森林模型来计算每个特征的重要性，随机森林是一种集成学习方法，它通过构建多个决策树并取平均值来提高模型的准确性和稳定性
2. 创建影子特征：为了评估原始特征的重要性，Boruta算法会对数据进行打乱，生成一组与原始特征对应的影子特征，这些影子特征是通过随机排列原始特征的数据生成的，它们不应该对目标变量有预测能力
3. 比较特征重要性：算法将原始特征的重要性与影子特征的重要性进行比较，如果一个原始特征的重要性显著高于所有影子特征的最大重要性，那么它被认为是“重要的”；如果它的重要性低于影子特征的最大重要性，则被认为是“无关的”，对于那些无法明显判断的重要性，算法会将其标记为“待定”
4. 逐步消除无关特征：算法通过迭代的方式，逐步剔除那些被标记为“无关”的特征，然后重新构建模型，直到所有特征都被分类为“重要”或“无关”，或者达到设定的迭代次数
5. 输出结果：最终，Boruta算法输出三类特征——重要的、无关的和待定的，重要特征可以保留用于模型构建，而无关特征则可以被舍弃，对于待定的特征，可能需要进一步的分析或选择性保留
```

In [ ]:
# Boruta特征选择：基于随机森林的Boruta算法应用
# 代码参考：https://mp.weixin.qq.com/s/wX1B_2wxp3kUVjQKQE5XLw
# 讲解参考：https://www.bilibili.com/video/BV1T5sVeMEGC/

# =================================================================================
# 1.数据读取处理
# 读取数据，将其分为特征（X）和目标变量（y），然后将数据集按80%训练集和20%测试集进行划分，使用的是一个心脏电复律的数据集包含46个特征变量一个目标变量为二分类任务
# =================================================================================
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

df = pd.read_csv('/home/phl/PHL/Car-T/model_v1/B-NHL_reindexed_example/csv/B-NHL_static_data.csv')
# 划分特征和目标变量
X = df.drop(['Infection'], axis=1)
y = df['Infection']
# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=df['Infection'])
df.head()

# =================================================================================
# 2.Boruta算法特征选择
# 初始化随机森林分类器模型用于评估特征重要性，并通过Boruta特征选择器与影子特征进行对比，确定哪些特征是重要的，随后对训练数据应用Boruta算法，以筛选出对模型预测最有用的特征，Boruta算法默认在内部迭代100次，如需更改迭代次数，可通过添加参数max_iter进行调整
# =================================================================================
from sklearn.ensemble import RandomForestClassifier
from boruta import BorutaPy
# 初始化随机森林模型
rf = RandomForestClassifier(n_jobs=-1, class_weight='balanced', max_depth=5)
# 初始化Boruta特征选择器
boruta_selector = BorutaPy(rf, n_estimators='auto', verbose=2, random_state=42)
# 对训练数据进行特征选择
boruta_selector.fit(X_train.values, y_train.values)
# 检查选中的特征
selected_features = X_train.columns[boruta_selector.support_].to_list()
# 打印被选择的特征
print("Selected Features: ", selected_features)
# 打印被剔除的特征
rejected_features = X_train.columns[~boruta_selector.support_].to_list()
print("Rejected Features: ", rejected_features)
# 打印有待定性的特征
tentative_features = X_train.columns[boruta_selector.support_weak_].to_list()
print("Tentative Features: ", tentative_features)


# 获取特征排名
# 使用boruta_selector.ranking_获取每个特征的排名，这个排名表示了特征的重要性，数值越小代表特征越重要，这里只展示部分特征，最后确定重要的特征为['Type_of_atrial_fibrillation', 'BMI', 'Left_atrial_diameter', 'Systolic_blood_pressure', 'NtproBNP']
# feature_ranks = boruta_selector.ranking_
# 将特征名称和排名结合成一个DataFrame
feature_importance_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Rank': feature_ranks
})
feature_importance_df


# =================================================================================
# 3.多次运行Boruta算法以评估特征排名稳定性
# 多次运行Boruta算法，以不同的随机种子生成特征排名，并将每次的排名结果保存到一个DataFrame中，用于分析特征排名的一致性，并且指定max_iter=50不在是默认的100，提高运行速度
# =================================================================================
# 初始化随机森林模型
rf = RandomForestClassifier(n_jobs=-1, class_weight='balanced', max_depth=5)

# 初始化存储特征排名的 DataFrame
ranking_df = pd.DataFrame(index=range(1, 21), columns=X_train.columns)

# 运行 Boruta 20 次
for i in range(20):
    print(f"Iteration {i+1}")
    
    # 初始化Boruta特征选择器
    boruta_selector = BorutaPy(rf, n_estimators='auto', verbose=2, random_state=i, max_iter=50)
    
    # 对训练数据进行特征选择
    boruta_selector.fit(X_train.values, y_train.values)
    
    # 获取特征排名
    feature_ranks = boruta_selector.ranking_
    
    # 将特征排名保存到 DataFrame 中
    ranking_df.loc[i+1] = feature_ranks
ranking_df

# =================================================================================
# 可视化排名稳定性
# 从图表中可以看出，最左侧的特征（Type_of_atrial_fibrillation、BMI等）在多次迭代中排名较高，表明它们是模型中较为重要的特征，而最右侧的特征（如COPD、Amiodarone等）则在多次迭代中排名较低，说明它们在模型中被认为不太重要，与前面单次运行Boruta算法的结果一致，但是相比于单次运行，通过这个可视化可以发现某些特征在个别迭代中的重要性排名与大多数迭代结果不同，从而判断这些特征的重要性是否稳定，图中位于须线外的点正是代表了这些异常的排名结果，如果过多这种情况出现作者认为需要着重去考虑这种特征
# =================================================================================
# 确保数据集中只有数值列
numeric_ranking_df = ranking_df.apply(pd.to_numeric, errors='coerce')

# 计算每个特征的中位数
median_values = numeric_ranking_df.median()

# 根据中位数对列进行排序
sorted_columns = median_values.sort_values().index

# 设置绘图风格
plt.figure(figsize=(15, 8))
sns.set(style="whitegrid")

# 绘制箱线图
sns.boxplot(data=numeric_ranking_df[sorted_columns], palette="Greens")

plt.xticks(rotation=90)
plt.title("Sorted Feature Ranking Distribution by Boruta", fontsize=16)
plt.xlabel("Attributes", fontsize=14)
plt.ylabel("Importance", fontsize=14)
plt.tight_layout()
plt.show()

# Borutashap

- 代码链接：https://github.com/Ekeany/Boruta-Shap/tree/master
- 对应含义：https://blog.csdn.net/gitblog_00754/article/details/155299350

In [ ]:
# =============================================================================
# 模块 5_Borutashap: Using Shap and Basic Random Forest
# =============================================================================

from BorutaShap import BorutaShap, load_data
  
X, y = load_data(data_type='regression')
X.head()

# no model selected default is Random Forest, if classification is True it is a Classification problem
Feature_Selector = BorutaShap(importance_measure='shap',
                              classification=False)

'''
Sample: Boolean
	if true then a rowise sample of the data will be used to calculate the feature importance values

sample_fraction: float
	The sample fraction of the original data used in calculating the feature importance values only
        used if Sample==True.

train_or_test: string
	Decides whether the feature improtance should be calculated on out of sample data see the dicussion here.
        https://slds-lmu.github.io/iml_methods_limitations/pfi-data.html

normalize: boolean
            if true the importance values will be normalized using the z-score formula

verbose: Boolean
	a flag indicator to print out all the rejected or accepted features.
'''
Feature_Selector.fit(X=X, y=y, n_trials=100, sample=False,
            	     train_or_test = 'test', normalize=True,
		     verbose=True)

# Returns Boxplot of features
Feature_Selector.plot(which_features='all')


# Returns a subset of the original data with the selected features
subset = Feature_Selector.Subset()


In [ ]:
# =============================================================================
# 模块 5_Borutashap: Using BorutaShap with another model XGBoost
# =============================================================================
from BorutaShap import BorutaShap, load_data
from xgboost import XGBClassifier

X, y = load_data(data_type='classification')
X.head()


model = XGBClassifier()

# if classification is False it is a Regression problem
Feature_Selector = BorutaShap(model=model,
                              importance_measure='shap',
                              classification=True)

Feature_Selector.fit(X=X, y=y, n_trials=100, sample=False,
            	     train_or_test = 'test', normalize=True,
		     verbose=True)

# Returns Boxplot of features
Feature_Selector.plot(which_features='all')

# Returns a subset of the original data with the selected features
subset = Feature_Selector.Subset()


## 超参数优化
https://cloud.baidu.com/article/5601757

### 一、超参数优化：机器学习模型性能提升的核心引擎
在机器学习模型开发过程中，超参数优化是连接算法理论与实际性能的关键桥梁。不同于通过数据训练自动调整的模型参数（如神经网络权重），超参数需在训练前预先设定，直接影响模型的学习能力与泛化性能。例如，决策树的深度、支持向量机的核函数类型、神经网络的学习率与层数等，均属于需要人工调优的超参数范畴。

超参数优化的本质是通过系统化探索超参数空间，寻找使模型在验证集或测试集上表现最优的参数组合。这一过程不仅能显著提升模型精度，还可避免因参数设置不当导致的过拟合或欠拟合问题。以图像分类任务为例，通过优化卷积神经网络的超参数（如滤波器数量、步长、正则化系数），模型在CIFAR-10数据集上的准确率可从75%提升至89%。

### 二、网格搜索：系统化穷举的经典方法
1. 原理与实现逻辑
网格搜索（Grid Search）通过定义超参数的候选值集合，构建所有可能的参数组合，并逐一评估模型性能。例如，若需优化学习率（取值为{0.01, 0.001, 0.0001}）和正则化系数（取值为{0.1, 0.01, 0.001}），则需训练3×3=9个模型，最终选择验证集上损失最小的组合。

2. 优缺点分析
- 优势：
    - 覆盖全面，避免遗漏潜在最优解
    - 实现简单，适用于参数空间较小（<10个参数）或离散值较多的场景
- 局限：
    - 计算成本随参数数量呈指数增长（如10个参数，每个取3个值，需评估3^10=59,049次）
    - 对连续参数需预先离散化，可能错过最优区间


### 四、贝叶斯优化：智能导向的先进方法
1. 原理与核心逻辑
贝叶斯优化（Bayesian Optimization）通过构建目标函数（如模型准确率）的概率代理模型（通常为高斯过程），结合采集函数（如Expected Improvement）动态选择下一个评估点。其核心优势在于“记忆性”：每次评估后更新代理模型，引导后续搜索向高潜力区域集中。

2. 优缺点分析
- 优势：
    - 计算效率极高，尤其适用于评估成本高昂的场景（如深度学习模型）
    - 自动平衡探索（新区域）与利用（已知高潜力区域）
- 局限：
    - 代理模型构建复杂，需调整超参数（如高斯过程的核函数）
    - 初始阶段可能因代理模型不准确而陷入局部最优


3. 方法：适用场景——计算效率——实现复杂度
- 网格搜索：低维离散参数空间——低——低
- 随机搜索：中高维参数空间，计算资源有限——中——中
- 贝叶斯优化：高维连续参数空间，评估成本高——高——高

In [ ]:
# 参考：https://cloud.baidu.com/article/5601757
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
# 定义参数搜索空间
search_spaces = {
    'C': Real(0.1, 10, prior='log-uniform'),
    'gamma': Real(1e-4, 1e-1, prior='log-uniform'),
    'kernel': Categorical(['rbf', 'poly']),
    'degree': Integer(2, 5)
}
# 创建贝叶斯优化对象（评估50次）
bayes_search = BayesSearchCV(
    SVC(), search_spaces, n_iter=50, cv=5, scoring='accuracy'
)
bayes_search.fit(X_train, y_train)
print("最优参数:", bayes_search.best_params_)

In [ ]:
# =============================================================================
# 参考：https://mp.weixin.qq.com/s/lvkn2PKv8AobZAsdpF9xjg
# 一、环境准备与可视化配置
# 首先，我们导入所需的库，并设置 Nature 风格配色 与绘图参数，以保证输出结果在学术论文与展示中具有顶刊水准。
# 这样配置后，模型结果图（如ROC曲线、混淆矩阵）会更符合国际期刊的展示风格。
# =============================================================================
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, roc_curve, confusion_matrix, f1_score
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import numpy as np
from skopt import BayesSearchCV
from skopt.space import Integer, Categorical, Real
import shap
import warnings
warnings.filterwarnings('ignore')

# 设置顶刊配色与字体
nature_colors = ['#E64B35', '#4DBBD5', '#00A087', '#3C5488', '#F39B7F']
sns.set_palette(nature_colors)
sns.set_style("whitegrid", {'grid.linestyle': '--', 'axes.edgecolor': '0.3'})
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['figure.dpi'] = 600
plt.rcParams['savefig.dpi'] = 600
plt.rcParams['savefig.format'] = 'jpeg'

# =============================================================================
# 📂 二、数据准备与划分
# 读取样本数据，提取特征和标签，并划分训练集与测试集。同时，将数据集导出，以便后续分析。
# 此步骤保证数据划分的独立性与可追溯性，避免数据结果不可复现。
# =============================================================================
# 导入CSV数据
data = pd.read_csv("sample.csv")

# 特征与标签
X = data.drop(columns=["label"])
y = data["label"]

# 划分训练集与测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# 导出数据
train_data = X_train.copy(); train_data['label'] = y_train
train_data.to_csv('train_dataset.csv', index=False)
test_data = X_test.copy(); test_data['label'] = y_test
test_data.to_csv('test_dataset.csv', index=False)

# =============================================================================
# 🔍 三、贝叶斯优化搜索空间设置
# 为了防止模型过拟合，我们在随机森林的参数空间中设置了较为保守的范围，并使用 BayesSearchCV 自动搜索最佳组合。
# 这里特别加入了 max_samples 与 ccp_alpha（复杂度剪枝），进一步提升模型的泛化性能。
# =============================================================================
# 定义超参数搜索空间
param_space = {
    'n_estimators': Integer(50, 200),     
    'max_depth': Integer(3, 15),          
    'min_samples_split': Integer(10, 30), 
    'min_samples_leaf': Integer(5, 20),   
    'max_features': Categorical(['sqrt', 'log2']),
    'bootstrap': Categorical([True]),     
    'max_samples': Real(0.6, 0.9),        
    'ccp_alpha': Real(0.0, 0.01)          
}

# =============================================================================
# ⚡ 四、BayesSearchCV超参数优化
# 通过贝叶斯优化+交叉验证，我们在合理的迭代次数内找到最佳参数组合，并保存优化过程。
# 这一步让随机森林在参数调优过程中避免了盲目搜索，提升了效率和模型的鲁棒性。
# =============================================================================
rf = RandomForestClassifier(random_state=42, n_jobs=-1)

# 执行BayesSearchCV
bayes_search = BayesSearchCV(
    estimator=rf,
    search_spaces=param_space,
    n_iter=100,
    cv=5,
    scoring='roc_auc',
    n_jobs=-1,
    random_state=42,
    verbose=1
)

bayes_search.fit(X_train, y_train)
print(f"最佳超参数: {bayes_search.best_params_}")
print(f"最佳交叉验证ROC AUC: {bayes_search.best_score_:.4f}")


# =============================================================================
# 📊 五、模型训练与性能评估
# 使用最佳参数训练模型，并在训练集与测试集上分别计算准确率、精确率、召回率、F1、ROC AUC等指标，同时进行过拟合分析。
# 通过对比训练集与测试集表现，可以直观判断模型是否存在过拟合。
# =============================================================================
# 使用最佳超参数训练模型
best_params = bayes_search.best_params_
rf_model = RandomForestClassifier(**best_params, random_state=42, n_jobs=-1)
rf_model.fit(X_train, y_train)

# 测试集预测
y_test_pred = rf_model.predict(X_test)
y_test_pred_proba = rf_model.predict_proba(X_test)[:, 1]
test_accuracy = accuracy_score(y_test, y_test_pred)
test_roc_auc = roc_auc_score(y_test, y_test_pred_proba)
print(f"Testing Accuracy: {test_accuracy:.4f}, ROC AUC: {test_roc_auc:.4f}")

# =============================================================================
# 📈 六、结果可视化：ROC与混淆矩阵
# 绘制训练集与测试集的 ROC曲线，并输出混淆矩阵，全面展示模型分类效果。
# =============================================================================
fpr_test, tpr_test, _ = roc_curve(y_test, y_test_pred_proba)
plt.figure(figsize=(8, 6))
plt.plot(fpr_test, tpr_test, color=nature_colors[1], lw=2.5, label=f'Test ROC (AUC={test_roc_auc:.4f})')
plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
plt.xlabel("False Positive Rate"); plt.ylabel("True Positive Rate")
plt.title("ROC Curve - Random Forest (Bayesian Optimized)")
plt.legend(); plt.savefig("ROC_RF_Bayes.jpeg", dpi=600); plt.show()

# =============================================================================
# 🌲 七、特征重要性与解释性分析（SHAP）
# 为了理解模型决策依据，我们不仅计算了随机森林自带的重要性，还结合 SHAP值 提供更细致的解释。
# 这一步不仅揭示了模型“如何判别滑坡”，还可以指导我们从地质机理角度验证模型合理性。
# =============================================================================
explainer = shap.TreeExplainer(rf_model)
shap_values = explainer.shap_values(X_test)

# 计算SHAP特征重要性
shap_importance = pd.DataFrame({
    'Feature': X.columns,
    'SHAP_Importance': np.abs(shap_values[1]).mean(axis=0)
}).sort_values('SHAP_Importance', ascending=False)

# 可视化前20个特征
sns.barplot(x='SHAP_Importance', y='Feature', data=shap_importance.head(20))
plt.xlabel("mean(|SHAP value|)")
plt.title("Top 20 SHAP Feature Importances")
plt.savefig("SHAP_RF_Bayes.jpeg", dpi=600)
plt.show()

# =============================================================================
# 八、模型预测：生成滑坡发生的概率值
# 第一步，我们需要利用训练好的随机森林模型对研究区的所有栅格样本进行预测。这里的重点是 读取全区特征数据 → 调用模型进行预测 → 输出每个栅格的滑坡概率。
# =============================================================================
def predict_probabilities():
    """使用随机森林模型进行预测并保存概率结果"""
    print("正在加载模型和数据进行预测...")
    try:
        # 加载优化后的随机森林模型
        RFmodel = joblib.load('RFmodel_anti_overfitting.pkl')
        
        # 加载全区样本数据
        test_data = pd.read_csv('tifftocsvxishui.csv').values
        # 用均值填充 NaN，保证预测过程不出错
        test_data = np.nan_to_num(test_data, nan=np.nanmean(test_data))
        
        # 计算每个样本的预测概率
        predicted_probabilities = RFmodel.predict_proba(test_data)
        
        # 保存概率结果
        probability_df = pd.DataFrame(predicted_probabilities, 
                                    columns=['Class_0_Probability', 'Class_1_Probability'])
        probability_df.to_csv('RFprob.csv', index=False)
        print("预测概率已保存为 RFprob.csv")
        
        return True
    except Exception as e:
        print(f"预测过程中出错: {str(e)}")
        return False

# =============================================================================
# 九、生成易发性栅格：空间化表达概率
# 有了每个样本点的概率值后，接下来需要将其映射回栅格空间。我们以一张参考栅格（如坡向 Aspect.tif）为模板，保证生成的易发性图与研究区空间范围和分辨率完全一致。
# =============================================================================
def create_susceptibility_raster():
    """将预测概率结果转换为易发性栅格"""
    print("正在生成易发性栅格...")
    try:
        # 读取预测结果，只取正类概率
        data = pd.read_csv('RFprob.csv')
        values = data['Class_1_Probability'].values

        # 打开参考栅格，读取空间信息
        with rasterio.open(r"J:\微信公众号\滑坡易发性评价模型\model\Aspect.tif") as src:
            profile = src.profile.copy()
            num_cols = src.width
            num_rows = src.height

        # 确认数据数量与栅格像元数一致
        if len(values) != num_rows * num_cols:
            raise ValueError(f"数据尺寸不匹配: CSV数据点({len(values)}) ≠ 栅格尺寸({num_rows*num_cols})")

        # 转换为二维数组并保存
        raster_data = values.reshape((num_rows, num_cols)).astype(np.float32)
        profile.update(dtype=rasterio.float32, count=1)
        
        output_path = r"J:\微信公众号\滑坡易发性评价模型进阶\易发栅格\BoCV-RF.tif"
        with rasterio.open(output_path, 'w', **profile) as dst:
            dst.write(raster_data, 1)
            
        print(f"易发性栅格已成功保存至 {output_path}")
        return True
    except Exception as e:
        print(f"生成栅格时出错: {str(e)}")
        return False

## 方法5:递归特征筛选
递归特征筛选（Recursive Feature Elimination, RFE）是一种常见的特征选择方法，其核心思想是基于模型自带的特征重要性排名，结合设定的步长和最低保留特征数量，对特征集进行逐步递归地筛选。在每次迭代中，根据模型计算的特征贡献度（如基于随机森林的Gini重要性或基于Boosting算法的分裂增益等），剔除一部分排名最低的特征，直到达到设定的最低特征数量或性能平衡点，该方法能够在减少特征数量的同时尽量保持模型性能的稳定性

In [ ]:
# =============================================================================
# 模块5.5————递归特征筛选实现特征选择
# 参考：# https://mp.weixin.qq.com/s/gfh5Ocv4ATpcMRbwNnJAAg
# =============================================================================

# =============================================================================
# 步骤一：数据准备
# =============================================================================
# 将数据集按特征变量 (X) 和目标变量 (y) 进行划分后，进一步将数据集分为训练集和测试集，确保目标变量 y 在训练集和测试集中类别分布比例一致（通过 stratify 参数）
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['axes.unicode_minus'] = False

df = pd.read_excel("2024-12-21公众号Python机器学习AI.xlsx")
# 划分特征和目标变量
X = df.drop(['y'], axis=1)
y = df['y']
from sklearn.model_selection import train_test_split

# 划分特征和目标变量
X = df.drop(['y'], axis=1)
y = df['y']
# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                    random_state=42, stratify=df['y'])
# =============================================================================
# 步骤二：递归特征消除（RFE）与多模型性能输出
# =============================================================================
'''
1.通过递归特征消减（RFE）方法，以步长为1逐步减少特征，计算不同特征数量下多个机器学习模型（RandomForest、GradientBoosting、XGBoost、LightGBM、CatBoost、AdaBoost）在测试集上的AUC分数，并记录每个模型在不同特征子集上的性能表现（最低特征数为1）。此外，除了使用此直接递归特征消减函数外，还存在结合交叉验证的递归特征消减（RFECV），能够更稳健地选择最佳特征数量，
2.K折交叉验证结合RFE与随机森林：特征选择全过程可视化参考：https://mp.weixin.qq.com/s?__biz=Mzk0NDM4OTYyOQ==&mid=2247486976&idx=1&sn=4096d77c1a9f704765bb8a7d3d290b2b&scene=21&poc_token=HMCBc2mj0lrZM0xfgLbTMbUgNmcx0JDyuZY0tsAH
'''
from sklearn.feature_selection import RFE
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

def perform_rfe_single_model(model, X_train, y_train, X_test, y_test, model_name):
    # 初始化RFE，设置所有特征为起始选择
    feature_count = len(X_train.columns)
    step = 1
    rfe = RFE(estimator=model, n_features_to_select=1, step=step)
    rfe.fit(X_train, y_train)
    # 提取特征数量和对应的特征排名
    rankings = rfe.ranking_
    sorted_indices = sorted(range(len(rankings)), key=lambda k: rankings[k])
    # 计算每个特征子集的ROC AUC分数（基于测试集）
    scores = []
    feature_counts = []
    for i in range(feature_count, 0, -step):
        selected_features_train = X_train.iloc[:, sorted_indices[:i]]
        selected_features_test = X_test.iloc[:, sorted_indices[:i]]
        
        # 在训练集上训练模型
        model.fit(selected_features_train, y_train)
        
        # 在测试集上进行预测并计算AUC
        y_pred = model.predict_proba(selected_features_test)[:, 1]
        score = roc_auc_score(y_test, y_pred)
        scores.append(score)
        feature_counts.append(i)
    results_df = pd.DataFrame({
        "Number_of_Features": feature_counts[::-1],  
        model_name: scores[::-1]                   
    })
    
    return results_df

models = {
    "RandomForest": RandomForestClassifier(random_state=42),
    "GradientBoosting": GradientBoostingClassifier(random_state=42),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42),
    "LightGBM": LGBMClassifier(random_state=42, verbose=-1),
    "CatBoost": CatBoostClassifier(verbose=0, random_state=42),
    "AdaBoost": AdaBoostClassifier(random_state=42, algorithm='SAMME')
}


results_df = None

# 遍历模型并运行RFE
for model_name, model in models.items():
    print(f"Running RFE for {model_name}...")
    model_results = perform_rfe_single_model(model, X_train, y_train, X_test, y_test, model_name)
    if results_df is None:
        results_df = model_results
    else:
        results_df = results_df.merge(model_results, on="Number_of_Features", how="outer")

results_df.reset_index(drop=True, inplace=True)
results_df

# =============================================================================
# 步骤三：基于递归特征消除的多模型AUC性能可视化与最佳特征选择
# =============================================================================
'''
1.通过筛选特征数量递减的模型性能数据，结合步长和初始值（当然这里的步长和初始值在执行RFE时就可以进行定义但是通过这种手段更加灵活而已），对多个机器学习模型在特征数量减少过程中的AUC变化进行可视化，同时绘制一条自定义的垂直线（optimal_features = 8）以标注自定义的最佳特征数量
2.需要注意的是，这里的 optimal_features = 8 并不是模型评价指标意义上的最优特征数量，而是出于实际需求（如特征数量较少但性能接近最优的可接受范围）的权衡结果。为了体现这一点，绘图时使用自定义的最佳特征数量，而不是严格依据AUC最高点的特征数量
'''
# 筛选出满足步长为1、初始值为1的行
step_size = 1  # 步长设置为1
initial_value = 1  # 初始值设置为1

# 筛选满足条件的数据行：特征数量 >= 初始值，并按步长取样
filtered_results_df = results_df[
    results_df["Number_of_Features"] >= initial_value  # 筛选特征数量大于等于初始值的行
].iloc[::step_size, :].reset_index(drop=True)  # 按步长取样，并重置索引

# 按 "Number_of_Features" 列从大到小排序
filtered_results_df.sort_values(by="Number_of_Features", ascending=False, inplace=True)

# 绘制图形
plt.figure(figsize=(8, 6))  # 设置画布大小
for column in filtered_results_df.columns[1:]:  # 遍历所有模型的列（从第2列开始）
    plt.plot(
        filtered_results_df["Number_of_Features"],  # X轴：特征数量
        filtered_results_df[column],               # Y轴：对应模型的AUC分数
        label=column,                              # 设置图例为模型名称
        marker='o',                                # 在曲线上标记点
        linewidth=1.5                              # 设置线条宽度
    )

# 绘制最佳特征数量的垂直虚线
optimal_features = 8  # 最佳特征数量
plt.axvline(
    x=optimal_features,  # 垂直线的位置
    color='black',       # 设置线的颜色为黑色
    linestyle='--',      # 设置线型为虚线
    label='Optimal Features'  # 图例说明
)

# 设置图表标题和坐标轴标签
plt.title('Feature Reduction (Step=7, Initial=1)', fontsize=14)  # 图表标题
plt.xlabel('Number of Features', fontsize=12)  # X轴标签
plt.ylabel('Area Under the ROC Curve (AUC)', fontsize=12)  # Y轴标签
# 设置X轴的刻度值和字体大小
plt.xticks(
    ticks=filtered_results_df["Number_of_Features"],  # 设置刻度值为筛选后的特征数量
    fontsize=10  # 设置字体大小
)
plt.yticks(fontsize=10)  # 设置Y轴字体大小
plt.legend(title="Models", fontsize=10, loc="best")  # 图例标题、字体大小及位置
plt.grid(axis='y', alpha=0.5)  # 添加Y轴方向的网格线，并设置透明度
plt.gca().spines['top'].set_visible(False)  # 隐藏顶部边框
plt.gca().spines['right'].set_visible(False)  # 隐藏右侧边框
plt.savefig('1.pdf', format='pdf', bbox_inches='tight', dpi=1200)
plt.tight_layout()
plt.show()

# =============================================================================
# 步骤四：递归特征消除：确定最佳模型与自定义特征选择
# =============================================================================
'''
# 当确定了最佳模型（如随机森林）以及自定义的最佳特征数量（如 feature_target = 8）时，由于该特征数量并非模型评价指标意义上的最优，而是根据实际需求自定义的结果，因此需要通过递归特征筛选的原理返回对应的特征集合，所以这里的perform_rfe_single_model为自定义函数用于返回模型所选择的特征数量
'''
# Random Forest 模型
random_forest_model = RandomForestClassifier(random_state=42)

# 执行 RFE，仅对 Random Forest
model_name = "RandomForest"
feature_target = 8
print(f"Running RFE for {model_name}...")

results_df, selected_features = perform_rfe_single_model(
    random_forest_model, X_train, y_train, X_test, y_test, model_name, feature_target
)

# 查看 Random Forest 的特征选择
print(f"{model_name} selected features when using {feature_target} features:")
print(selected_features)